In [ ]:
#Fig. 2A & Fig. S5
#Step 1: Open the total.net/maximum_spanning_tree file in 'D:Fig_2' with pajek 
#Step 2: using the Kamada Kawai layout algorithm to draw the network

In [ ]:
import pandas as pd
import math
import matplotlib.pyplot as plt
import os
import networkx as nx
import numpy as np

In [ ]:
#find the common neighbors of components in the original network data
file_path='D:Fig_2/original_data/'
files=os.listdir(file_path)
files.sort(key=lambda x:int(x.split('/')[-1].split('_')[0]))

for file in files:
    position=file_path+file
    df=pd.read_csv(position,encoding='gbk')
    df.set_index('herb1',inplace=True)
    df.columns.name='herb2'
    G=nx.Graph()
    idx=list(df.columns)
    G.add_nodes_from(idx)
    for i in range(len(idx)):
        for j in range(len(idx)):
            if df.loc[idx[i],idx[j]] != 0:
                G.add_edge(idx[i],idx[j])
    new_df=df.copy()
    new_df[new_df>0]=0
    for i in range(len(idx)):
        for j in range(len(idx)):
            if idx[i]!=idx[j]:
                new_df.loc[idx[i],idx[j]]=len(list(nx.common_neighbors(G,idx[i],idx[j])))
    new_df.to_csv('D:Fig_2/original_common_neighbor/'+file,encoding='gbk')

#calculate the NODF value of the original network
file_path1='D:Fig_2/original_data/'
file_path2='D:Fig_2/original_common_neighbor/'
files1=os.listdir(file_path1)
files1.sort(key=lambda x:int(x.split('/')[-1].split('_')[0]))
times=[]
NODF=[]
for s in range(len(files1)):
    print(files1[s])
    time=files1[s].replace('.csv','')
    position1=file_path1+files1[s]
    position2=file_path2+files1[s]
    df1=pd.read_csv(position1,encoding='gbk')
    df1.set_index('herb1',inplace=True)
    df1[df1>0]=1
    degree=list(df1.apply(lambda x:x.sum(),axis=0))
    df2=pd.read_csv(position2,encoding='gbk')
    df2.set_index('herb1',inplace=True)

    NR2=[]
    for i in range(len(df2)):
        NR1=[]
        for j in range(len(df2)):
            if j > i:
                if degree[j] == 0:
                    NR1.append(0)
                else:
                    NR1.append(df2.iloc[i,j]/degree[j])
        sum_r=sum(NR1)
        NR2.append(sum_r)
    NODF.append((sum(NR2)*2)/(len(df2)*(len(df2)-1)))
    times.append(time)
new_df=pd.DataFrame()
new_df['time']=times
new_df['NODF']=NODF
new_df.to_csv('D:Fig_2/original_nestedness.csv',index=False,encoding='gbk')

#acquire the null model of the original network
file_path='D:Fig_2/original_data/'
files=os.listdir(file_path)
files.sort(key=lambda x:int(x.split('/')[-1].split('_')[0]))
file_path1='D:Fig_2/null_model_data/'
for i in range(len(files)):
    print(files[i].replace('.csv',''))
    G=nx.Graph()
    position=file_path+files[i]
    df=pd.read_csv(position,encoding='gbk')
    df.set_index('herb1',inplace=True)
    
    for s in range(len(df)):
        for t in range(len(df)):
            if s > t:
                df.iloc[s,t]=0
    herbs=list(df.index)
    
    G.add_nodes_from(herbs)
    for s in range(len(df)):
        for t in range(len(df)):
            if df.iloc[s,t] != 0:
                G.add_edge(herbs[s],herbs[t])
    number_n=len(G.nodes())
    number_e=len(G.edges())
    total_n=number_n*(number_n-1)/2

    #construct the random network
    while True:
        R_G=nx.erdos_renyi_graph(number_n,number_e/total_n)
        if len(R_G.edges()) == number_e:
            break
        else:
            continue
    new_df=pd.DataFrame(np.random.rand(number_n*number_n).reshape(number_n,number_n),index=herbs,columns=herbs)
    new_df[new_df>0]=0
    e=list(R_G.edges())
    for item in e:
        for m in range(len(herbs)):
            if m == item[0]: 
                for n in range(len(herbs)):
                    if n == item[1]:
                        new_df.loc[herbs[m],herbs[n]] = 1
    new_df.to_csv(file_path1+files[i],encoding='gbk')

#find the common neighbors of components in the random network data
file_path='D:Fig_2/null_model_data/'
files=os.listdir(file_path)
files.sort(key=lambda x:int(x.split('/')[-1].split('_')[0]))

for file in files:
    position=file_path+file
    df=pd.read_csv(position,encoding='gbk')
    df.set_index('herb1',inplace=True)
    df.columns.name='herb2'
    G=nx.Graph()
    idx=list(df.columns)
    G.add_nodes_from(idx)
    for i in range(len(idx)):
        for j in range(len(idx)):
            if df.loc[idx[i],idx[j]] != 0:
                G.add_edge(idx[i],idx[j])
    new_df=df.copy()
    new_df[new_df>0]=0
    for i in range(len(idx)):
        for j in range(len(idx)):
            if idx[i]!=idx[j]:
                new_df.loc[idx[i],idx[j]]=len(list(nx.common_neighbors(G,idx[i],idx[j])))
    new_df.to_csv('D:Fig_2/null_model_common_neighbor/'+file,encoding='gbk')

#sort the null model data and null model common neighbor data according to degrees of components
#null model data
file_path='D:Fig_2/null_model_data/'
files=os.listdir(file_path)
files.sort(key=lambda x:int(x.split('/')[-1].split('_')[0]))
for s in range(len(files)):
    position=file_path+files[s]
    df=pd.read_csv(position,encoding='gbk')
    df.set_index('herb',inplace=True)
    herb=list(df.columns)
    dg=list(df.apply(lambda x:x.sum(),axis=1))
    dict1={}
    for i in range(len(herb)):
        dict1[herb[i]]=dg[i]
    dict2=sorted(dict1.items(),key=lambda x:x[1],reverse=True)
    new_herb=[]
    for item in dict2:
        new_herb.append(item[0])
    df=df.loc[new_herb,new_herb]
    df.to_csv(position,encoding='gbk')

#null model common neighbor data
file_path='D:Fig_2/null_model_data/'
files=os.listdir(file_path)
files.sort(key=lambda x:int(x.split('/')[-1].split('_')[0]))
file_path1='D:Fig_2/null_model_common_neighbor/'
for s in range(len(files)):
    position=file_path+files[s]
    df=pd.read_csv(position,encoding='gbk')
    df.set_index('herb',inplace=True)
    herb=list(df.columns)
    position1=file_path1+files[s]
    df1=pd.read_csv(position1,encoding='gbk')
    df1.set_index('herb',inplace=True)
    df1=df1.loc[herb,herb]
    df1.to_csv(position1,encoding='gbk')

#calculate the NODF value of the random network
file_path1='D:Fig_2/null_model_data/'
file_path2='D:Fig_2/null_model_common_neighbor/'
files1=os.listdir(file_path1)
files1.sort(key=lambda x:int(x.split('/')[-1].split('_')[0]))
times=[]
NODF=[]
for s in range(len(files1)):
    print(files1[s])
    time=files1[s].replace('.csv','')
    position1=file_path1+files1[s]
    position2=file_path2+files1[s]
    df1=pd.read_csv(position1,encoding='gbk')
    df1.set_index('herb',inplace=True)
    degree=list(df1.apply(lambda x:x.sum(),axis=0))
    df2=pd.read_csv(position2,encoding='gbk')
    df2.set_index('herb',inplace=True)

    NR2=[]
    for i in range(len(df2)):
        NR1=[]
        for j in range(len(df2)):
            if j > i:
                if degree[j] == 0:
                    NR1.append(0)
                else:
                    NR1.append(df2.iloc[i,j]/degree[j])
        sum_r=sum(NR1)
        NR2.append(sum_r)
    NODF.append((sum(NR2)*2)/(len(df2)*(len(df2)-1)))
    times.append(time)
new_df=pd.DataFrame()
new_df['time']=times
new_df['NODF']=NODF
new_df.to_csv('D:Fig_2/null_model_nestedness.csv',index=False,encoding='gbk')

In [ ]:
#plot Fig.2B
df1=pd.read_csv('D:Fig_2/nestedness/original_nestedness.csv',encoding='gbk')
df2=pd.read_csv('D:Fig_2/nestedness/null_model_nestedness.csv',encoding='gbk')

time=list(df1['time'])
NODF_1=list(df1['NODF'])
NODF_2=list(df2['NODF'])

fig = plt.figure(figsize=(12,6))
ax1=plt.subplot(1,1,1)
ax1.plot(time,NODF_1,c='green',marker='^',label='TCM')
ax1.plot(time,NODF_2,c='red',marker='*',label='null model')
ax1.set_xlabel('time',fontsize=15)
ax1.set_ylabel('NODF',fontsize=15)
ax1.set_xticklabels(time, rotation=90,fontsize=15)
ax1.set_yticklabels([0.00,0.00,0.05,0.10,0.15,0.20,0.25,0.30,0.35], fontsize=15)
ax1.legend(loc='upper center', bbox_to_anchor=(0.5, 1.1))
ax1.axvline(0,ls="--",linewidth=0.3,c="black")
ax1.axvline(1.2,ls="--",linewidth=0.3,c="black")
ax1.axvline(4.81,ls="--",linewidth=0.3,c="black")
ax1.axvline(8.6,ls="--",linewidth=0.3,c="black")
ax1.axvline(12.68,ls="--",linewidth=0.3,c="black")
ax1.axvline(15.44,ls="--",linewidth=0.3,c="black")
ax1.axvline(18.11,ls="--",linewidth=0.3,c="black")
ax1.axvline(19,ls="--",linewidth=0.3,c="black")
ax1.text(-0.45,0.27,'before the\nend of Han\n Dynasty',fontsize=15)
ax1.text(2.1,0.26,'Wei,Jin,\nSouthern and\nNorthern\nDynasties'.center(3),fontsize=15)
ax1.text(6.1,0.27,'Sui,Tang\nand Five\nDynasties'.center(8),fontsize=15)
ax1.text(10.1,0.27,'Song,Jin\nand Yuan\nDynasties'.center(11),fontsize=15)
ax1.text(13.3,0.27,'Ming\nDynasty'.center(14),fontsize=15)
ax1.text(16.2,0.27,'Qing\nDynasty'.center(16),fontsize=15)
ax1.text(18.,0.27,'Modern\nTimes'.center(-5),fontsize=15)
font1 = { 'size': 15 }
plt.legend(loc='lower center',bbox_to_anchor=(0.8, 0.03),ncol=2,prop=font1 )
plt.savefig('D:Fig_2/compare_nestedness.TIFF',dpi=300,bbox_inches='tight')
plt.show()

In [ ]:
#plot Fig.3C
import pandas as pd
import seaborn as sns
sns.set()

df=pd.read_csv('D:Fig_2/original_data/1901_2016.csv',encoding='gbk')
df.set_index('herb1',inplace=True)
df[df>1]=1
fig = plt.figure(figsize=(6,5))
ax=plt.subplot(1,1,1)
sns.heatmap(df, xticklabels=False, yticklabels=False,cmap='binary')  
ls=list(df.apply(lambda x: x.sum(), axis=1))
x=[]
for i in range(len(df)):
    x.append(i+1)
ax.plot(x,ls,color='orangered')
ax.set_xlabel('components',fontsize=15)
ax.set_ylabel('components',fontsize=15)
ax.set_title('TCM(1901-2016)',fontsize=15)
plt.savefig('D:Fig_2/original_1901_2016.TIFF',dpi=300,bbox_inches='tight')
plt.show()

In [ ]:
#plot Fig.3D
import pandas as pd
import seaborn as sns
sns.set()
df=pd.read_csv('D:Fig_2/null_model_data/1901_2016.csv',encoding='gbk')
fig = plt.figure(figsize=(6,5))
ax=plt.subplot(1,1,1)
sns.heatmap(df, xticklabels=False, yticklabels=False,cmap='binary')  
ax.set_xlabel('components',fontsize=15)
ax.set_ylabel('components',fontsize=15)
ax.set_title('null model(1901-2016)',fontsize=15)
plt.savefig('D:Fig_2/null_model_1901_2016.TIFF',dpi=300,bbox_inches='tight')
plt.show()